# Atividade 3: Classificação de Lixo Doméstico

> Classificação de lixo doméstico utilizando Python e Keras.

## Desafio

Classificar alguns objetos encontrados em lixo doméstico usando o _dataset_ do Kaggle disponível em https://www.kaggle.com/datasets/farzadnekouei/trash-type-image-dataset/.
O conjunto de dados possui 6 classes (6 tipos de lixo):

- 📦 Caixas de papelão;
- 🥂 Vidro;
- 🛢️ Metal;
- 🗞️ Papel;
- 🥤 Plástico;
- 🗑️ Entulhos (restos de embalagem, comida e outros que não se enquadram nas categorias anteriores).

## Autores

- Orientadora: Elloá B. Guedes - [@elloa](https://github.com/elloa)
- Time:
  - Debora Souza Barros - [@Debby-Barros](https://github.com/Debby-Barros)
  - Diana Martins - [@ddianaom](https://github.com/ddianaom)
  - Gabriel Dos Santos Lima - [@gabrielSantosLima](https://github.com/gabrielSantosLima)
  - Thiago Marques - [@tmmarquess ](https://github.com/tmmarquess)


## Etapa 0: Configuração do ambiente

Os tópicos que serão abordados nesta etapa:
* Importação das bibliotecas
* Baixar o _dataset_ para o arquivo local do projeto  

In [ ]:
!pip install optuna keras_tuner tensorflow[and-cuda] kaggle

In [ ]:
import os
import zipfile
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import keras_tuner as kt

from collections import Counter
from glob import glob
from keras_tuner import HyperModel
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
# baixando do kaggle
if not os.path.isdir('dataset'):
  !rm -r sample_data
  !kaggle datasets download -d farzadnekouei/trash-type-image-dataset
  !unzip trash-type-image-dataset.zip
  !rm trash-type-image-dataset.zip
  !mv TrashType_Image_Dataset dataset
else:
  print("Conjunto de dados já existe no diretório atual.")

## Etapa 1: Importação do conjunto de dados

Os tópicos que serão abordados nesta etapa:
* Importar o _dataset_
* Verificar quantos exemplos o _dataset_ possui

In [ ]:
# diretório do dataset
base_dir = 'dataset'

# quantidade de exemplos do dataset
image_files = glob(os.path.join(base_dir, '**', '*.jpg'), recursive=True)
print(f'O dataset possui {len(image_files)} imagens')

## Etapa 2: Análise exploratória

Os tópicos que podem ser abordados nesta etapa:
* Buscar explorar informações relevantes sobre a base de dados. Algumas sugestões de perguntas que podem servir como ponto de partida:
  * Quantas classes existem?
  * Quantos exemplos cada classe possui?
* Analisar a qualidade das imagens do _dataset_ e descrever as limitações que podem ser encontradas (se possível apresentar exemplos)

In [ ]:
# quantidade de classes no dataset
count_classes = 0
for dir in os.listdir(base_dir):
  count_classes += 1

print(f"No dataset 'Trash type' existem {count_classes} classes")

In [ ]:
# quantidades de exemplos em cada classe
files_count = {}
for root, dirs, files in os.walk(base_dir):
  for dir in dirs:
    qtd_files = os.path.join(root, dir)
    count = len(os.listdir(qtd_files))
    files_count[dir] = count


for key, item in files_count.items():
  print(f'Na classe "{key}" existem {item} imagens')

In [ ]:
# Dimensões das imagens
def img_dimensions(img_dir):
    files = os.listdir(img_dir)
    dim = []
    for file in files:
        img_path = os.path.join(img_dir, file)
        img = cv2.imread(img_path)

        height, width, channels = img.shape
        dim.append((height, width))

    count_dim = Counter(dim)

    print("Dimensões mais comuns:")
    for dim, freq in count_dim.most_common(15):
        print(f"Dimensão (altura x largura): {dim}, Frequência: {freq}")
    print('\n')


for dir in os.listdir(base_dir):
  dir_path = os.path.join(base_dir, dir)
  if os.path.isdir(dir_path):
    print(f'Analisando imagens em: {dir_path}')
    img_dimensions(dir_path)

In [ ]:
# plotando algumas imagens das classes do dataset 'Trash Type'
def plot_images_from_subfolders(base_dir, num_images=3):
    subfolders = [os.path.join(base_dir, folder) for folder in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, folder))]

    for folder_path in subfolders:
        print(f"Imagens de: {folder_path}")
        fig, axes = plt.subplots(nrows=1, ncols=num_images, figsize=(15, 5))
        files = os.listdir(folder_path)

        for i in range(num_images):
            img_path = os.path.join(folder_path, files[i])
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            axes[i].imshow(img_rgb)
            axes[i].axis('off')
        plt.show()

plot_images_from_subfolders(base_dir, 3)

## Etapa 3: Pré-processamento

Os tópicos que podem ser abordados nesta etapa:
* Definir o tamanho da grade de busca a ser contemplada
* Preparar o conjunto de dados para o treinamento com a estratégia de validação cruzada _holdout_

In [ ]:
# Utilizano Keras
param_grid_keras = {
    'units': [32, 64, 128, 256],
    'activation': ['relu', 'tanh', 'sigmoid'],
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'learning_rate': [0.01, 0.1, 0.4],
    'batch_size': [32, 64],
    'epochs': [10, 20, 30]
}

In [ ]:
# Preparar o conjunto de dados para o treinamento com a estratégia de validação cruzada holdout
data = []
labels = []

for root, dirs, files in os.walk(base_dir):
  for dir in dirs:
    for file in os.listdir(os.path.join(root, dir)):
      img_path = os.path.join(root, dir, file)
      img = cv2.imread(img_path)
      img = np.array(img)
      label = dir
      data.append(img)
      labels.append(label)

In [ ]:
# Organizando algumas informações sobre o conjunto de dados
num_classes = 6
image_shape = data[0].shape

In [ ]:
# Preparando a transformação dos rótulos para atributos categóricos utilizando OneHotEncoder
encoder = LabelEncoder()

In [ ]:
X = np.array(data)
y = to_categorical(encoder.fit_transform(np.array(labels)))

x_train_temp, x_test, y_train_temp, y_test = train_test_split(X, y, test_size=.3, shuffle=True) # Holdout 70/30
x_train, x_val, y_train, y_val = train_test_split(x_train_temp, y_train_temp, test_size=.2, shuffle=True) # Holdout 80/20

## Etapa 4: Treinamento e testes dos modelos

Os tópicos que serão abordados nesta etapa:
* Definir qual o modelo que será utilizado e quais arquiteturas serão avaliadas
* Preparar modelo(s) para grade de busca
* Treinamento
* Teste do(s) modelo(s)

In [ ]:
class CNNModel(HyperModel):
    def __init__(self, input_shape, num_classes, name=None, tunable=True):
       super().__init__(name, tunable)
       self.input_shape = input_shape
       self.num_classes = num_classes

    def __choice_param(self, param, hp):
      return hp.Choice(param, param_grid_keras[param])

    def build(self, hp):
        model = keras.models.Sequential()

        # Input Layer
        model.add(keras.layers.Input(shape=self.input_shape))

        # Feature Layers
        model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation=self.__choice_param('activation', hp)))
        model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation=self.__choice_param('activation', hp)))
        model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(keras.layers.Conv2D(128, kernel_size=(3, 3), activation=self.__choice_param('activation', hp)))
        model.add(keras.layers.Conv2D(128, kernel_size=(3, 3), activation=self.__choice_param('activation', hp)))
        model.add(keras.layers.GlobalAveragePooling2D())
        model.add(keras.layers.Dropout(0.5))

        # Dense Layers
        model.add(keras.layers.Dense(128, activation=self.__choice_param('activation', hp)))
        model.add(keras.layers.Dense(self.num_classes, activation='softmax'))

        # Preparing model to train
        model.compile(loss = 'categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(learning_rate=self.__choice_param('learning_rate', hp)),
                      metrics=['accuracy'])
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=self.__choice_param('batch_size', hp),
            **kwargs,
        )

### Treinando

In [ ]:
epochs = 1
max_trials = 5
early_stopping_callback = EarlyStopping(monitor='val_loss',
                                        patience=3)

Treinamento: Rede customizada

In [ ]:
tuner_custom = kt.RandomSearch(
    CNNModel(image_shape, num_classes),
    objective='val_accuracy',
    directory='models/custom',
    overwrite=True,
    max_trials=max_trials)

In [ ]:
tuner_custom.search(
    x_train,
    y_train,
    epochs=epochs,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping_callback])

Treinamento: ResNet

In [ ]:
batch_size = 32

In [ ]:
tuner_resnet = kt.RandomSearch(
    kt.applications.HyperResNet(input_shape=image_shape, classes=num_classes),
    objective='val_accuracy',
    directory='models/resnet',
    overwrite=True,
    max_trials=max_trials)

In [ ]:
tuner_resnet.search(
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping_callback])

### Recuperando os melhores modelos

In [ ]:
def get_best_model(tuner):
  best_models = tuner.get_best_models(num_models=1)
  return best_models[0]

In [ ]:
custom_best_model = get_best_model(tuner_custom)
custom_best_model.summary()

In [ ]:
resnet_best_model = get_best_model(tuner_resnet)
resnet_best_model.summary()

In [ ]:
def print_history_of_model(model):
  history = model.fit(
      x_train,
      y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data=(x_val, y_val))

  plt.figure(figsize=(6,6))
  plt.plot(history.history['accuracy'], label='acurácia do treinamento')
  plt.plot(history.history['val_accuracy'], label='acurácia da validação')
  plt.title('Histórico de Acurácia')
  plt.xlabel('Épocas')
  plt.ylabel('Acurácia')
  plt.legend()
  plt.show()

In [ ]:
print_history_of_model(custom_best_model)

In [ ]:
print_history_of_model(resnet_best_model)

### Salvando os modelos

In [ ]:
custom_best_model.save('model_custom.keras')

In [ ]:
resnet_best_model.save('model_resnet.keras')

## Etapa 5: Análise quantitativa e qualitativa de desempenho dos modelos avaliados

Os tópicos que podem ser abordados nesta etapa:
* Análise quantitativa do(s) modelo(s)
* Análise qualitativa do(s) modelo(s)
* Conclusão. Incluir na dissertação:
  * Sugestões de melhoria;
  * Desafios;
  * Próximos passos.

### Avaliando a qualidade dos modelos

In [ ]:
# Função de avaliação
def show_metrics(y_true, y_pred):
    # Matriz de Confusão
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    plt.title('Matriz de Confusão')
    sns.heatmap(cm, annot=True, fmt='.0f', cmap='Blues')
    plt.show()

    # Acurácia
    acc = accuracy_score(y_true, y_pred)
    print(f"\nAcurácia: {acc:.4f}")

    # F1-Score
    f_score = f1_score(y_true, y_pred, average='weighted')
    print(f"F1-Score: {f_score:.4f}")

    # Precisão
    precision = precision_score(y_true, y_pred, average='weighted')
    print(f"Precisão: {precision:.4f}")

    # Revocação
    recall = recall_score(y_true, y_pred, average='weighted')
    print(f"Revocação: {recall:.4f}")

def load_and_predict(model_path, x_test):
    model = keras.models.load_model(model_path)
    y_pred = model.predict(x_test)
    return np.argmax(y_pred, axis=1)

def evaluate_model(model_name, y_test_classes, y_pred):
    print(f"Métricas do {model_name}:")
    show_metrics(y_test_classes, y_pred)

In [ ]:
# Carregar e prever usando os modelos
y_pred_custom = load_and_predict('model_custom.keras', x_test)
y_pred_resnet = load_and_predict('model_resnet.keras', x_test)

# Converter y_test para classes
y_test_classes = np.argmax(y_test, axis=1)

In [ ]:
evaluate_model("Modelo Custom", y_test_classes, y_pred_custom)

In [ ]:
evaluate_model("Modelo ResNet", y_test_classes, y_pred_resnet)